# IVR7 Advanced 2 Example
This example aims to highlight the follow:
* illustrate the usage of the data-upload as well as model versioning.
* Builds a simple pickup/dropoff (similar to the basic model).
* Configures "open routing" - i.e. do not cost the return legs for the vehicles to the end-location
* Illustrates how to reference matrix data in a (potentially) versioned model.
* or load overriding elements on a particular matrix directly.
  * one can provide a complete matrix in this manner too if needed!

## Requirements
This example assumes you've configured an api-key with the required services enabled (see the [portal](portal.icepack.ai) for configuration details) and that you're familiar with loading and working with protobuf models.

## Data
We're going to load sample data which has order sizes and durations in a tabular format.


In [ ]:
import pandas
df = pandas.read_csv('../../sample_data/publist_orders.csv')
print(df.head())

## Model Configuration
This model builds the ivr7-basic example.
The same dimensions (distance,time and capacity) are used as well as a handful of jobs, one vehicle cost class, one vehicle class and five vehicles.

**note**: we're going to add an extra location called "vehicle-site" to the model. This is going to allow us to accurately control the distances between locations, differentiating between those at the pickup site (Guiness storehouse) and the vehicle end-site ('vehicle-site')

In [ ]:
exec(open('apiHelper.py').read()) # import some api-helper classes which we've written for you.
exec(open('ivr7-model-helper.py').read()) # import some modelling helpers

api = apiHelper(modelType="ivr7-kt461v8eoaif") # set the model type to the ivr7 model

sr = ivr7_kt461v8eoaif_pb2.SolveRequest()
sr.solveType = 0 # optimise solve request

m = sr.model
m.dimensions.CopyFrom(make_distance_time_cap_dims())
m.locations.extend(make_locations(df)) 
m.locations.append(m.locations[0]) # just copy the Guiness storehouse location
m.locations[len(m.locations) - 1].id = 'vehicle-site' #rename it and we're good to go (location id's need to be unique in a model)
m.jobs.extend(make_job_time_cap(df, [0] * (df.shape[0]-1), range(1, df.shape[0])))
m.vehicleCostClasses.append(make_vcc_simple('vcc1', 1000, 1.001e-2, 1.001e-2, 1.001e-2, 1, 3))
m.vehicleClasses.append(make_vc_simple('vc1', 1, 1, 1, 1))
for i in range(1,5):
    m.vehicles.append( make_vehicle_cap('vehicle_' + str(i), 'vc1', 'vcc1',
                                        2000, # the vehicle capacity
                                        'vehicle-site', # the start location
                                        'vehicle-site', # the end location
                                        7*60,        # the start time (7AM)
                                        18*60))       # the end time  (6PM)
# at this point we have a complete model which we can continue to manipulate

### Manipulating the model
We've created vehicles which need to start/end at "vehicle-site". Now we can make the last change which is to override the distance between locations and the "vehicle-site". we're only going to modify the distances FROM locations TO "vehicle-site". If you wanted to do complete line-haul outsourcing-style modelling, you could also do this for "vehicle-site" TO all alocations. For now, we'll just demonstrate the open routing case (i.e. the last leg is not costed). This is more common in practice as you'll try to find hauliers which start close to the footprint you're modelling.

You have two ways of doing this. Upload it via the data-api, or upload it as part of the model.
Feel free to modify the value of `dataUpload` to understand the two different code-paths.


### Configuration a cached model
At this point we've built and run a basic model. Lets now use the objects, but submit them to the api through a different mechanism.

In [ ]:
dataUpload = True

if dataUpload:
    data_api = apiHelper(modelType="ivrdata-o43e0dvs78zq") # this instantiates a second model helper, 
                                                           # one which works with data payloads
    data_model = ivrdata_o43e0dvs78zq_pb2.CachedTransitSet()
    for i in range(0, len(m.locations) - 1):
        tv = ivrdata_o43e0dvs78zq_pb2.TransitSet.TransitValue()
        tv.fromId = m.locations[i].id
        tv.toId = 'vehicle-site'
        tv.value = 0
        data_model.transitSet.transits.append(tv)
    modelID = data_api.Post(data_model) # send the transit sets to the api
    print("data model reference: " + modelID)
else:
    ts = ivr7_kt461v8eoaif_pb2.TransitSet()
    for i in range(0, len(m.locations) - 1):
        tv = ivr7_kt461v8eoaif_pb2.TransitSet.TransitValue()
        tv.fromId = m.locations[i].id
        tv.toId = 'vehicle-site'
        tv.value = 0
        ts.transits.append(tv)
    print(ts.transits[1:5]) # just display the first five
    
    

tgen_d = ivr7_kt461v8eoaif_pb2.TransitGenerator()
tgen_d.id = 'custom_distance'
tgen_t = ivr7_kt461v8eoaif_pb2.TransitGenerator()
tgen_t.id = 'custom_time'

# now depending on the code path, we either upload this using the modelID or the transit set in the model
if dataUpload:
    tgen_d.requestId = modelID # link this to the data we uploaded earlier
    tgen_t.requestId = modelID # we can do this because it's actually the same data (a column of zeros)
else:
    tgen_d.transitSet.CopyFrom(ts)
    tgen_t.transitSet.CopyFrom(ts)

del m.transitGenerators[:] # in case you want to run this twice
m.transitGenerators.extend([tgen_d, tgen_t]) # so this adds the custom transit generators to the model

#print(m.transitGenerators)


### Linking up the custom transits
The last step required to link up the custom transits is to give a heads up to the vehicle classes that they should apply these custom-transits. We're going to keep the existing road-network transit generator in the list of attributes, but we're going to add an additional attribute for the `distance` and `time` dimension. The order is important here, the solver will pull through the attributes in the order provided for each dimension, so first the road-network matrix, then the custom matrix data. This allows us to effectively update the road-network data with our custom transits.

In [ ]:
# first the custom time 
t_attr = ivr7_kt461v8eoaif_pb2.VehicleClass.Attribute()
t_attr.dimensionId = 'time'
t_attr.transitGeneratorId = 'custom_time'
t_attr.transitCoef = 1.0
t_attr.locationCoef = 1.0
t_attr.taskCoef = 1.0
m.vehicleClasses[0].attributes.append(t_attr)

d_attr = ivr7_kt461v8eoaif_pb2.VehicleClass.Attribute()
d_attr.dimensionId = 'distance'
d_attr.transitGeneratorId = 'custom_distance'
d_attr.transitCoef = 1.0
m.vehicleClasses[0].attributes.append(d_attr)

print(m.vehicleClasses)


### Send the model to the api
We can now submit the model to the api.

In [ ]:
reqId = api.Post(sr)
sol = api.Get(reqId)
t = tabulate(sr, sol)


### Checking the output
We can confirm the effect of the custom distane and time on the output of the model. Lets focus on the transit edges that connect the last node to the vehicle-end sites. These values should be zero if we expressed things correctly to the api:

In [ ]:
edges = t['edges']
edges['distance_delta'] = edges['distance_end'] - edges['distance_end']
edges['time_delta'] = edges['time_end'] - edges['time_end']
w = edges['toLocationId'] == 'vehicle-site'
edges[['fromStopId', 'toStopId', 'fromLocationId', 'toLocationId','distance_delta', 'time_delta', 'distance_start', 
      'distance_end', 'time_start', 'time_end']][w]

# and there we have it. The distance and time deltas on the last two legs from the last 
# location to the vehicle-site are zero

### Visualise the response
Now we can visualise this. We woudl expect to see that the last stop on a route is quite far from the depot because the stops which are farthest away from all the other nodes are typically the points at which the route should end (because you're saving the most in terms of distance and time).


In [ ]:
from ipyleaflet import Map, Polyline, Circle, LayerGroup

cvec = list(['blue', 'red', 'orange', 'green', 'black'])
cdict = {}
for i in range(0,5):
    cdict['vehicle_' + str(i)] = cvec[i]


# form some locations to add to the map (just green dots seems ok)
locs = list()
for index, p in t['nodes'].iterrows():
    circle = Circle()
    circle.location = (p['y'], p['x'])
    circle.radius = 10
    circle.color = "green"
    locs.append(circle)

center = [df['Y'][1],df['X'][1]] # just use the first point as the center (i.e. the depot)
m = Map(scroll_wheel_zoom=True, center=center, zoom=12)
for i, gs in enumerate(t['edges']['geometry']):
    if t['edges']['toLocationId'][i] != 'vehicle-site': # we don't want to plot these edges
        m.add_layer(Polyline(locations=gs,color=cdict[t['edges']['vehicleId'][i]],fill=False))

m.add_layer(LayerGroup(layers=(locs)))
m

### What next?
We suggest you have a look at the IVR8 examples on how to use compartments (should your model require this degree of modelling).